# TESTE PRÁTICO DE LINGUAGEM DE CONSULTA ESTRUTURADA (SQL)
Para a realização das atividades, você deverá executar as instruções abaixo para importar as bibliotecas e funções necessárias para criar a base de dados.

Ao final, esperamos que você desenvolva os códigos selecionados e entregue o notebook com as consultas SQL desenvolvidas e o arquivo .db criado.

O teste importará três arquivos cruciais para o desenvolvimento dessa atividade. Todos estão no Githun (não mude as URL's).

*   usuarios.csv
*   produtos.csv
*   vendas.csv

## O teste foi desenvolvido para ser executado com o SQLite3 e no Google Colab (Ubuntu 18.04). Você deverá utilizar consultas no padrão ANSI. Os comandos necessários podem ser encontrados em:

https://www.sqlite.org/docs.html

https://www.tutorialspoint.com/sqlite/index.htm



# AVISO!



1.   Execute o código abaixo para atualizar a biblioteca do SQLite3 utilizada para essa atividade
2.   **Ignore as mensagens de erro sobre perda de conectividade na instrução abaixo (o kernel será reiniciado para atualização do SQLite3)**


In [ ]:
import os

# BAIXAR A ÚLTIMA RELEASE DO COLAB, DESCOMPACTAR E INSTALAR NO DIRETÓRIO DO 
# PYTHON 3.7 (VERSÃO USADA PELO COLAB NESSE NOTEBOOK)
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content/sqlite/
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

# REINICIANDO O KERNEL DO COLAB PARA IMPORTAR A BIBLIOTECA ATUALIZADA
os.kill(os.getpid(), 9)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  9250k      0  0:00:01  0:00:01 --:--:-- 9250k
/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for a sed that does not truncate output... /bin/sed
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for fgrep... /bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checkin

# INSTRUÇÕES PARA REALIZAR AS CONSULTAS
### PARA CONSULTAR DADOS, SIGA OS PASSOS ABAIXO:

#### Abrir conexão:
    conexao_sql = sqlite3.connect(banco)
#### Abrir cursor para executar consulta no banco:
    cursor = conexao_sql.cursor()
#### Criar uma consulta:
    consulta = '''
        Coloque aqui a sua instrução SQL;
    '''
    cursor.execute(consulta)
#### Para visualizar o resultado da sua consulta:
    for linha in cursor:
      print(linha)
#### Para fechar a conexão:
    conexao_sql.close()

### LEMBRE-SE DE QUE PYTHON É SENSÍVEL A IDENTAÇÃO DE CÓDIGO!

# IMPORTANDO BIBLIOTECAS NECESSÁRIAS PARA EXECUTAR OS CÓDIGOS EM SQL

In [1]:
import sqlite3
import csv
import requests
from pathlib import Path
from urllib.parse import urlparse

# DEFINIÇÃO DE VARIÁVEIS FIXAS PARA O PROJETO

In [2]:
banco = "teste_sql.db"

# FUNÇÃO PARA INSERIR REGISTROS NO BANCO DE DADOS

In [3]:
# Recebe como parâmetro
# 1. Nome do banco de dados
# 2. Nome da tabela que receberá os dados
# 3. Lista das colunas da tabela
# 4. Dados que serão inseridos

def inserir_registros(banco, tabela, lista_colunas, arquivo):
  conexao_sql = sqlite3.connect(banco)
  cursor = conexao_sql.cursor()
  dados = open(arquivo)
  conteudo_dados = csv.reader(dados)
  num_colunas = 1 if lista_colunas.count(',') == 1 else lista_colunas.count(',')
  string_insert = 'INSERT INTO ' + tabela + ' (' + lista_colunas + ') VALUES (?' + ('' if num_colunas == 1 else ', ?' * (num_colunas)) + ')'
  cursor.executemany(string_insert, conteudo_dados)
  conexao_sql.commit()
  conexao_sql.close()

# FUNÇÃO PARA BAIXAR A BASE DE DADOS UTILIZADA PARA DESENVOLVIMENTO DAS ATIVIDADES

In [4]:
# Recebe como parâmetro:
# 1. Nome do arquivo com a extensão (Ex: base.csv)
# 2. URL do site onde o dado está localizado (Ex: 'https://drive.google.com/user_teste/folder_dataset/file.csv'):

def baixar_dados(nome_arquivo, url):
  requisicao = requests.get(url)
  conteudo = requisicao.content
  arquivo_csv = open(nome_arquivo, 'wb')
  arquivo_csv.write(conteudo)
  arquivo_csv.close()

# CRIAÇÃO DE CONEXÃO COM O BANCO SQLITE3 PARA DESENVOLVIMENTO DA ATIVIDADE

In [5]:
# Abre conexão com o banco. Se o arquivo não existir, cria a base.
conexao_sql = sqlite3.connect(banco)
conexao_sql.close()

# CRIAÇÃO DAS TABELAS PARA REALIZAÇÃO DAS ATIVIDADES

### TABELA DE USUÁRIOS

In [6]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/usuarios.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS usuarios
(
  cod_usuario INTEGER NOT NULL,
  data_cadastro DATE NOT NULL,
  faixa_etaria TEXT,
  cidade TEXT NOT NULL,
  estado TEXT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, data_cadastro, faixa_etaria, cidade, estado'
inserir_registros(banco, 'usuarios', lista_campos, nome_arquivo)

### TABELA DE PRODUTOS

In [7]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/produtos.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_produtos = 'produtos.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS produtos
(
  cod_produto INTEGER NOT NULL,
  nome_produto TEXT NOT NULL,
  categoria_produto TEXT,
  valor_produto FLOAT);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_produto, nome_produto, categoria_produto, valor_produto'
inserir_registros(banco, 'produtos', lista_campos, arquivo_produtos)

### TABELA DE VENDAS

In [8]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/vendas.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_vendas = 'vendas.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS vendas
(
  cod_usuario INTEGER NOT NULL,
  cod_produto INTEGER NOT NULL,
  data_compra DATE NOT NULL,
  quantidade INT NOT NULL,
  valor FLOAT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, cod_produto, data_compra, quantidade, valor'
inserir_registros(banco, 'vendas', lista_campos, arquivo_vendas)

In [9]:
def consulta(query):
  conexao_sql = sqlite3.connect(banco)
  cursor = conexao_sql.cursor()
  consulta = query
  cursor.execute(consulta)
  for linha in cursor:
    print(linha)
  conexao_sql.close()

# PERGUNTA 1:

### Escreva um comando em SQL que retorne a quantidade de vendas dos estados de Minas Gerais e São Paulo por ano e mês

In [10]:
consulta('''
    SELECT 
    strftime('%Y', data_compra) AS ANO
    ,strftime('%m', data_compra) AS MES
    ,count(*) 
    FROM vendas v
    INNER JOIN usuarios u ON(
      v.cod_usuario = u.cod_usuario
    )
    WHERE estado IN ('Minas Gerais', 'São Paulo')
    GROUP BY strftime('%Y', data_compra), strftime('%m', data_compra)
''')

('2018', '06', 1)
('2018', '07', 1)
('2018', '08', 1)
('2018', '10', 2)
('2018', '11', 1)
('2018', '12', 2)
('2019', '01', 2)
('2019', '02', 1)
('2019', '04', 2)
('2019', '05', 1)
('2019', '06', 2)
('2019', '07', 2)
('2019', '08', 2)
('2019', '09', 2)
('2019', '10', 1)
('2019', '11', 1)
('2019', '12', 3)
('2020', '01', 2)
('2020', '02', 1)
('2020', '03', 2)
('2020', '04', 2)
('2020', '05', 3)
('2020', '06', 4)
('2020', '07', 2)
('2020', '09', 5)
('2020', '10', 5)
('2020', '11', 3)
('2020', '12', 1)
('2021', '01', 2)
('2021', '02', 7)
('2021', '03', 2)
('2021', '04', 9)
('2021', '05', 6)
('2021', '06', 9)
('2021', '07', 16)


# PERGUNTA 2:

### Escreva um comando em SQL que retorne o ticket médio (valor médio de compra) e quantidade de compras realizadas dos usuários que fizeram compras entre 2019 e 2020 e que tiveram mais de 3 compras no período (lembre-se de contar todas as compras dos usuários que estão nessa condição!)

In [20]:
consulta('''
    SELECT v.cod_usuario
    ,avg(valor) AS ticket
    ,count(*) AS qtd_vendas
    FROM vendas v
    INNER JOIN usuarios u ON(
      v.cod_usuario = u.cod_usuario
    )
    WHERE strftime('%Y', data_compra) IN ('2020','2019')
    GROUP BY v.cod_usuario
    HAVING count(*) > 3
''')

(69, 25926.0925, 4)
(89, 22791.025, 4)
(176, 1686.6919999999998, 5)
(191, 1971.8649999999998, 4)
(441, 2147.8666666666663, 6)
(700, 21861.0, 6)
(796, 1696.875, 4)
(898, 3710.4750000000004, 4)
(1074, 21402.79714285714, 7)
(1192, 2565.625, 4)
(1415, 210.55499999999998, 6)
(1696, 252.87833333333336, 6)
(1752, 13832.692857142858, 7)
(1756, 24269.69, 5)
(1787, 15756.61, 7)
(1947, 14866.7325, 4)
(2183, 1482.16, 5)
(2340, 22284.579999999998, 6)
(2392, 8185.0650000000005, 4)
(2438, 17550.012, 5)
(2551, 17265.393333333333, 6)
(2597, 8681.897500000001, 4)
(2646, 12171.340000000002, 5)
(2669, 6578.05, 4)
(2811, 20773.65, 4)
(2843, 16535.013750000002, 8)
(2856, 13890.898000000001, 5)
(2920, 15125.482, 5)
(2981, 21811.9, 5)
(3090, 13108.057142857142, 7)
(3358, 27187.438000000002, 5)
(3403, 14547.94875, 8)


# PERGUNTA 3:

### Escreva um comando em SQL que retorne quais são os usuários por estado que mais compraram em todo o período analisado e, o número de compras realizadas, a quantidade total de itens comprados e valor total pago

In [12]:
consulta('''
    SELECT estado 
    ,v.cod_usuario
    ,count(*) AS qtd_vendas
    ,sum(quantidade) AS qtd_total_itens
    ,sum(valor) AS valor_total
    FROM vendas v
    INNER JOIN usuarios u ON(
      v.cod_usuario = u.cod_usuario
    )
    GROUP BY estado, v.cod_usuario
''')

('Amapá', 2186, 7, 61, 51359.34)
('Bahia', 176, 5, 43, 8433.46)
('Bahia', 466, 4, 33, 13613.890000000001)
('Bahia', 2155, 4, 39, 13820.400000000001)
('Bahia', 2856, 7, 71, 71986.05)
('Bahia', 2981, 9, 70, 126282.4)
('Bahia', 3400, 5, 35, 7052.29)
('Ceará', 1787, 8, 79, 121494.67)
('Goiás', 700, 9, 74, 142389.09)
('Goiás', 1878, 4, 54, 96862.1)
('Goiás', 2850, 5, 64, 107155.5)
('Goiás', 3358, 7, 58, 136881.19)
('Maranhão', 191, 7, 69, 26216.35)
('Maranhão', 441, 8, 95, 14945.129999999997)
('Maranhão', 2811, 13, 117, 184101.88999999998)
('Mato Grosso', 1192, 7, 76, 16001.5)
('Mato Grosso', 2340, 9, 112, 165197.3)
('Mato Grosso do Sul', 679, 6, 102, 152271.96)
('Mato Grosso do Sul', 2492, 3, 20, 17998.0)
('Minas Gerais', 69, 5, 53, 104538.31)
('Minas Gerais', 124, 2, 27, 4454.83)
('Minas Gerais', 479, 4, 51, 5820.95)
('Minas Gerais', 796, 7, 56, 28161.0)
('Minas Gerais', 1947, 6, 70, 62296.630000000005)
('Minas Gerais', 2009, 6, 53, 24173.87)
('Minas Gerais', 2216, 7, 75, 85841.41)
('Mina

# PERGUNTA 4:

### Escreva um comando em SQL que retorne quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga

In [13]:
consulta('''
    SELECT v.cod_produto
    ,nome_produto
    ,sum(quantidade) AS qtd_total_itens
    ,sum(valor) AS valor_total
    ,avg(valor) AS media
    FROM vendas v
    INNER JOIN usuarios u ON(
      v.cod_usuario = u.cod_usuario
    )
    INNER JOIN produtos p ON(
      v.cod_produto = p.cod_produto
    )
    WHERE estado IN ('Paraná', 'Rio Grande do Sul', 'Santa Catarina', 'Espírito Santo', 'Minas Gerais', 'Rio de Janeiro', 'São Paulo')
    GROUP BY estado, v.cod_usuario
    ORDER BY COUNT(*) DESC
    LIMIT 3
''')

(2, 'Bicicleta Caloi Aro 29', 114, 137228.01, 13722.801000000001)
(2, 'Bicicleta Caloi Aro 29', 93, 95697.3, 10633.033333333333)
(7, 'Ar-condicionado 9500 BTUs LG', 76, 29114.449999999997, 3234.9388888888884)


# PERGUNTA 5:

### Escreva um comando em SQL que retorne o ticket médio e o número total de usuários que fizeram pelo menos uma compra por faixa etária

In [14]:
consulta('''
  SELECT v.cod_usuario
  ,avg(valor) AS ticket
  ,count(*) AS num_usuarios
  FROM vendas v
  INNER JOIN usuarios u ON(
    v.cod_usuario = u.cod_usuario
  )
  GROUP BY v.cod_usuario
  HAVING count(*) > 1
''')

(28, 3234.9388888888884, 9)
(69, 20907.662, 5)
(89, 15337.894999999999, 6)
(124, 2227.415, 2)
(153, 15910.335, 4)
(156, 4303.49, 3)
(176, 1686.6919999999998, 5)
(185, 663.55, 2)
(191, 3745.192857142857, 7)
(441, 1868.1412499999997, 8)
(464, 12259.349999999999, 6)
(466, 3403.4725000000003, 4)
(479, 1455.2375, 4)
(679, 25378.66, 6)
(700, 15821.01, 9)
(796, 4023.0, 7)
(813, 20053.384, 5)
(898, 3078.1600000000003, 5)
(999, 4710.365, 6)
(1074, 17685.02, 9)
(1192, 2285.9285714285716, 7)
(1415, 430.80428571428575, 7)
(1676, 3718.2600000000007, 5)
(1691, 8627.220000000001, 6)
(1696, 252.87833333333336, 6)
(1752, 12128.55625, 8)
(1756, 16801.625, 8)
(1769, 14751.908333333335, 6)
(1787, 15186.83375, 8)
(1796, 18229.899999999998, 3)
(1878, 24215.525, 4)
(1947, 10382.771666666667, 6)
(1974, 27999.494, 5)
(2009, 4028.978333333333, 6)
(2034, 36957.926666666666, 6)
(2155, 3455.1000000000004, 4)
(2183, 1344.9183333333333, 6)
(2186, 7337.0485714285705, 7)
(2216, 12263.058571428572, 7)
(2274, 984.35, 2)

# PERGUNTA 6:

### Escreva um comando em SQL que aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado

In [ ]:
consulta('''
  
''')

# PERGUNTA 7

### Escreva um comando em SQL que retorna o produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago

In [15]:
consulta('''
  SELECT p.nome_produto
  ,count(*) as num_usuarios
  ,min(p.valor_produto)
  ,max(p.valor_produto)
  FROM vendas v
  INNER JOIN produtos p ON(
    v.cod_produto = p.cod_produto
  )
  GROUP BY v.cod_usuario
  HAVING count(*) > 1
''')

('Smartphone Samsung', 4, 5.0, 2049.5)
('SPA Intel 700 L', 13, 4.99, 5999.1)
('SPA Intel 700 L', 6, 138.99, 5999.1)
('Cafeteira Nespresso', 4, 4.99, 219.57)
('Mochila Targus', 4, 4.99, 58.9)
('SPA Intel 700 L', 13, 4.99, 5999.1)
('SmartTV Samsung 4K 55 polegadas', 6, 58.9, 2499.9)
('SPA Intel 700 L', 5, 58.9, 5999.1)
('SPA Intel 700 L', 5, 5.0, 5999.1)
('Ar-condicionado 9500 BTUs LG', 4, 9.9, 1300.49)
('Notebook Asus Intel Core i7', 7, 7.9, 4590.9)
('Microoondas 35L', 5, 4.99, 389.1)
('Notebook Asus Intel Core i7', 5, 7.9, 4590.9)
('SmartTV Samsung 4K 55 polegadas', 5, 5.0, 2499.9)
('Bicicleta Caloi Aro 29', 7, 4.99, 1345.78)
('SPA Intel 700 L', 11, 4.99, 5999.1)
('SmartTV Samsung 4K 55 polegadas', 9, 4.99, 2499.9)
('Escrivaninha em L', 4, 4.99, 437.0)
('SmartTV Samsung 4K 55 polegadas', 5, 5.0, 2499.9)
('Smartphone Samsung', 7, 5.0, 2049.5)
('Notebook Asus Intel Core i7', 7, 4.99, 4590.9)
('SPA Intel 700 L', 9, 4.99, 5999.1)
('Bicicleta Caloi Aro 29', 10, 4.99, 1345.78)
('Bicicleta Ca

# PERGUNTA 8

### Escreva um comando em SQL que retorne o total de compras realizadas, o total de itens comprados e a receita total obtida

In [16]:
consulta('''
    SELECT COUNT(*) AS qtd_compras 
    ,v.cod_usuario
    ,count(*) AS qtd_vendas
    ,sum(quantidade) AS qtd_total_itens
    ,sum(valor) AS valor_total
    FROM vendas v
    INNER JOIN usuarios u ON(
      v.cod_usuario = u.cod_usuario
    )
    GROUP BY estado, v.cod_usuario
''')

(7, 2186, 7, 61, 51359.34)
(5, 176, 5, 43, 8433.46)
(4, 466, 4, 33, 13613.890000000001)
(4, 2155, 4, 39, 13820.400000000001)
(7, 2856, 7, 71, 71986.05)
(9, 2981, 9, 70, 126282.4)
(5, 3400, 5, 35, 7052.29)
(8, 1787, 8, 79, 121494.67)
(9, 700, 9, 74, 142389.09)
(4, 1878, 4, 54, 96862.1)
(5, 2850, 5, 64, 107155.5)
(7, 3358, 7, 58, 136881.19)
(7, 191, 7, 69, 26216.35)
(8, 441, 8, 95, 14945.129999999997)
(13, 2811, 13, 117, 184101.88999999998)
(7, 1192, 7, 76, 16001.5)
(9, 2340, 9, 112, 165197.3)
(6, 679, 6, 102, 152271.96)
(3, 2492, 3, 20, 17998.0)
(5, 69, 5, 53, 104538.31)
(2, 124, 2, 27, 4454.83)
(4, 479, 4, 51, 5820.95)
(7, 796, 7, 56, 28161.0)
(6, 1947, 6, 70, 62296.630000000005)
(6, 2009, 6, 53, 24173.87)
(7, 2216, 7, 75, 85841.41)
(8, 2438, 8, 91, 95031.67)
(4, 2903, 4, 25, 8965.169999999998)
(3, 2908, 3, 31, 6708.820000000001)
(4, 2910, 4, 40, 58935.54000000001)
(9, 3090, 9, 93, 95697.3)
(5, 3248, 5, 72, 132507.8)
(5, 898, 5, 48, 15390.800000000001)
(6, 2392, 6, 42, 41412.0900000000

# PERGUNTA 9

### Escreva um comando em SQL que retorne o produto, a quantidade de itens vendidos e a receita do produto que tem mais itens vendidos por ano e mês

In [17]:
consulta('''
    SELECT DISTINCT u.*
    FROM usuarios u
    LEFT JOIN vendas v ON(
      v.cod_usuario = u.cod_usuario
    )
    WHERE v.cod_usuario IS NULL
''')

('cod_usuario', 'data_cadastro', 'faixa_etaria', 'cidade', 'estado')
(5098, '2021-05-03', 'Entre 62 a 70 anos', 'Guaraci', 'Paraná')
(5229, '2019-11-22', 'Entre 10 a 15 anos', 'Cerqueira César', 'São Paulo')
(5482, '2020-09-22', 'Entre 62 a 70 anos', 'Bebedouro', 'São Paulo')
(5504, '2019-09-01', 'Entre 16 a 21 anos', 'Cariri do Tocantins', 'Tocantins')
(5614, '2020-11-02', 'Entre 50 a 61 anos', 'Itaguaçu da Bahia', 'Bahia')
(5923, '2019-09-12', 'Entre 16 a 21 anos', 'Coremas', 'Paraíba')
(6985, '2019-09-12', 'Entre 62 a 70 anos', 'Brejetuba', 'Espírito Santo')
(7901, '2019-09-28', 'Entre 16 a 21 anos', 'Itaguaí', 'Rio de Janeiro')
(9980, '2021-02-09', 'Entre 62 a 70 anos', 'Sátiro Dias', 'Bahia')
(10080, '2020-09-21', 'Entre 10 a 15 anos', 'Rio Sono', 'Tocantins')
(10152, '2019-10-29', 'Entre 62 a 70 anos', 'General Carneiro', 'Mato Grosso')
(10170, '2020-09-18', 'Entre 16 a 21 anos', 'Juazeiro', 'Bahia')
(10929, '2021-04-10', 'Entre 37 a 49 anos', 'Brejo dos Santos', 'Paraíba')
(1100

# PERGUNTA 10

### Escreva um comando em SQL que exiba quem são os usuários que compraram produtos diferentes

In [18]:
consulta('''
    SELECT v.cod_usuario, count(DISTINCT cod_produto)
    FROM vendas v
    GROUP BY v.cod_usuario
''')

(1, 4)
(2, 10)
(3, 4)
(4, 4)
(5, 2)
(6, 10)
(7, 5)
(8, 5)
(9, 5)
(10, 4)
(11, 7)
(12, 5)
(13, 5)
(14, 5)
(15, 6)
(16, 9)
(17, 6)
(18, 4)
(19, 5)
(20, 7)
(21, 7)
(22, 8)
(23, 6)
(24, 6)
(25, 6)
(26, 6)
(27, 3)
(28, 8)
(29, 4)
(30, 7)
(31, 11)
(32, 5)
(33, 4)
(34, 7)
(35, 2)
(36, 5)
(37, 8)
(38, 3)
(39, 9)
(40, 6)
(41, 8)
(42, 6)
(43, 6)
(44, 3)
(45, 6)
(46, 6)
(47, 5)
(48, 8)
(49, 4)
(50, 8)
(51, 6)
(52, 3)
(53, 10)
(54, 6)
(55, 8)
(56, 6)
(57, 5)
(58, 3)
(59, 6)
(60, 3)
(61, 4)
(62, 6)
(63, 5)
(64, 7)
(65, 5)
(66, 5)
(67, 3)
(68, 8)
(69, 5)
(70, 5)
(71, 5)
(72, 4)
(73, 4)
(74, 7)
(75, 6)
(76, 6)
(77, 4)
(78, 7)
(79, 4)
(80, 6)
(81, 4)
(82, 2)
(83, 4)
(84, 6)
(85, 4)
(86, 7)
(87, 4)
(88, 8)
(89, 5)
(90, 4)
(91, 7)
(92, 4)
(93, 5)
(94, 5)
(95, 5)
(96, 2)
(97, 6)
(98, 4)
(99, 4)
(100, 5)
(101, 6)
(102, 2)
(103, 6)
(104, 4)
(105, 3)
(106, 7)
(107, 5)
(108, 1)
(109, 6)
(110, 7)
(111, 4)
(112, 3)
(113, 7)
(114, 6)
(115, 6)
(116, 5)
(117, 3)
(118, 4)
(119, 5)
(120, 5)
(121, 7)
(122, 4)
(123, 

# PERGUNTA 11

### Escreva um comando em SQL que verifique usuários que não fizeram nenhuma compra

In [19]:
consulta('''
    SELECT DISTINCT u.*
    FROM usuarios u
    LEFT JOIN vendas v ON(
      v.cod_usuario = u.cod_usuario
    )
    WHERE v.cod_usuario IS NULL
''')

('cod_usuario', 'data_cadastro', 'faixa_etaria', 'cidade', 'estado')
(5098, '2021-05-03', 'Entre 62 a 70 anos', 'Guaraci', 'Paraná')
(5229, '2019-11-22', 'Entre 10 a 15 anos', 'Cerqueira César', 'São Paulo')
(5482, '2020-09-22', 'Entre 62 a 70 anos', 'Bebedouro', 'São Paulo')
(5504, '2019-09-01', 'Entre 16 a 21 anos', 'Cariri do Tocantins', 'Tocantins')
(5614, '2020-11-02', 'Entre 50 a 61 anos', 'Itaguaçu da Bahia', 'Bahia')
(5923, '2019-09-12', 'Entre 16 a 21 anos', 'Coremas', 'Paraíba')
(6985, '2019-09-12', 'Entre 62 a 70 anos', 'Brejetuba', 'Espírito Santo')
(7901, '2019-09-28', 'Entre 16 a 21 anos', 'Itaguaí', 'Rio de Janeiro')
(9980, '2021-02-09', 'Entre 62 a 70 anos', 'Sátiro Dias', 'Bahia')
(10080, '2020-09-21', 'Entre 10 a 15 anos', 'Rio Sono', 'Tocantins')
(10152, '2019-10-29', 'Entre 62 a 70 anos', 'General Carneiro', 'Mato Grosso')
(10170, '2020-09-18', 'Entre 16 a 21 anos', 'Juazeiro', 'Bahia')
(10929, '2021-04-10', 'Entre 37 a 49 anos', 'Brejo dos Santos', 'Paraíba')
(1100